In [1]:
import openeo
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
# Establish the connetion to openeo
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [10]:
# Set up the region of interest
name = 'Stromboli'
vent_location = vlat, vlon = [38.789, 15.213]
clear_locations = [
    [38.789, 16.213],
    [39.589, 16.213],
    [39.589, 15.213],
    [39.589, 14.213],
    [38.789, 14.213]
]
bounds = lonW, lonE, latS, latN = [13.7, 16.6, 38, 40]
radius = 25000

start_date = '2018-06-01'
end_date = '2020-01-01'

In [3]:
# SHV
name = 'SHV'
vent_location = vlat, vlon = [16.710289, -62.175658]
clear_locations = [
    [17.710289, -62.175658],
    [17.710289, -61.175658],
    [16.710289, -61.175658],
    [15.710289, -61.175658],
    [15.710289, -62.175658]
]
bounds = lonW, lonE, latS, latN = [-64, -59, 15, 18]
radius = 25000

start_date = '2018-06-01'
end_date = '2025-10-01'

In [3]:
# Hayli Gubbi
name = 'HayliGubbi'
bounds = lonW, lonE, latS, latN = [36.7, 44.7, 9.5, 17.5]

start_date = '2025-09-01'
end_date = '2025-11-22'

In [3]:
# Whakaari
name = 'Whakaari'
bounds = lonW, lonE, latS, latN = [172.2, 182, -40.5, -34.5]

start_date = '2018-06-01'
end_date = '2025-11-01'

In [4]:
# Define the datacube to acquire
so2_cube = connection.load_collection(
    "SENTINEL_5P_L2",
    temporal_extent=(start_date, end_date),
    spatial_extent={
        "west": lonW, "south": latS, "east": lonE, "north": latN,
        "crs": "EPSG:4326"
    },
    bands=["SO2"]
)

# Now aggregate by day to avoid having multiple data per day
so2_cube = so2_cube.aggregate_temporal_period(reducer="mean", period="day")

job = so2_cube.execute_batch(
    title=f"SO2 from {name}",
    outputfile=f"{name}_{start_date}_{end_date}_so2_data.nc"
)

0:00:00 Job 'j-2511251141354830bb3d7697e19cc231': send 'start'
0:00:13 Job 'j-2511251141354830bb3d7697e19cc231': created (progress 0%)
0:00:18 Job 'j-2511251141354830bb3d7697e19cc231': created (progress 0%)
0:00:25 Job 'j-2511251141354830bb3d7697e19cc231': created (progress 0%)
0:00:33 Job 'j-2511251141354830bb3d7697e19cc231': running (progress N/A)
0:00:43 Job 'j-2511251141354830bb3d7697e19cc231': running (progress N/A)
0:00:55 Job 'j-2511251141354830bb3d7697e19cc231': running (progress N/A)
0:01:10 Job 'j-2511251141354830bb3d7697e19cc231': running (progress N/A)
0:01:30 Job 'j-2511251141354830bb3d7697e19cc231': running (progress N/A)
0:01:54 Job 'j-2511251141354830bb3d7697e19cc231': running (progress N/A)
0:02:24 Job 'j-2511251141354830bb3d7697e19cc231': running (progress N/A)
0:03:01 Job 'j-2511251141354830bb3d7697e19cc231': running (progress N/A)
0:03:48 Job 'j-2511251141354830bb3d7697e19cc231': running (progress N/A)
0:04:46 Job 'j-2511251141354830bb3d7697e19cc231': running (progr

In [4]:
# Define the datacube to acquire
so2_cube = connection.load_collection(
    "TERRASCOPE_S5P_L3_SO2CBR_TD",
    temporal_extent=(start_date, end_date),
    spatial_extent={
        "west": lonW, "south": latS, "east": lonE, "north": latN,
        "crs": "EPSG:4326"
    },
    bands=["SO2"]
)

# Now aggregate by day to avoid having multiple data per day
so2_cube = so2_cube.aggregate_temporal_period(reducer="mean", period="day")

job = so2_cube.execute_batch(
    title=f"SO2 from {name}",
    outputfile=f"{name}_{start_date}_{end_date}_cobra_data.nc"
)

OpenEoApiError: [404] CollectionNotFound: Collection 'TERRASCOPE_S5P_L3_SO2CBR_TD' does not exist. (ref: r-25111910104842839f3f9c3733b1f7fa)